In [1]:
!pip install tensorflow-model-optimization

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import sys

2025-12-15 19:37:32.258007: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 19:37:32.259425: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 19:37:32.286955: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-15 19:37:32.287549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-15 19:37:32.888125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [3]:
base_path = "/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition"
base_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition'

In [4]:
wisdm_dataset_path = f'{base_path}/data/WISDM'
wisdm_dataset_path

'/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/data/WISDM'

In [5]:
if base_path not in sys.path:
    sys.path.append(base_path) 

In [6]:
import src.load_wisdm_dataset as wisdm

In [7]:
Xtrain, ytrain, xtest, ytest, labels = wisdm.load_wisdm_dir(wisdm_dataset_path)

In [8]:
import src.get_models as models

In [9]:
import importlib
importlib.reload(models)

<module 'src.get_models' from '/s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/src/get_models.py'>

In [10]:
base_model = keras.models.load_model(f'{base_path}/models/gru_model_wisdm.keras')

2025-12-15 19:38:32.592511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-15 19:38:32.619572: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-12-15 19:38:32.733314: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gra

In [11]:
baseline_model_loss, baseline_model_accuracy = base_model.evaluate(
    xtest, ytest)

print('Baseline test accuracy:', baseline_model_accuracy)


2025-12-15 19:38:35.573457: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:38:35.574213: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:38:35.574977: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

110/110 [==============================] - 1s 9ms/step - loss: 0.7990 - accuracy: 0.8411
Baseline test accuracy: 0.8411480784416199


In [12]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

batch_size = 64
epochs = 100
validation_split = 0.1

num_images = Xtrain.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(base_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 6)]          0         
                                                                 
 prune_low_magnitude_gru_2 (  (None, 128, 64)          27267     
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_gru_3 (  (None, 32)               18627     
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_dense_2  (None, 128)              8322      
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_batch_n  (None, 128)              513       
 ormalization_1 (PruneLowMag                               

2025-12-15 19:38:43.219121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:38:43.220009: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:38:43.220594: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [13]:
callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=f'{base_path}/pruned_models_wisdm_gru'),
]

model_for_pruning.fit(Xtrain, ytrain,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/100


2025-12-15 19:38:53.871191: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:38:53.872329: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:38:53.872971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

191/192 [============================>.] - ETA: 0s - loss: 0.0790 - accuracy: 0.9752

2025-12-15 19:39:03.858341: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:39:03.859248: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:39:03.859872: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

192/192 [==============================] - 11s 43ms/step - loss: 0.0789 - accuracy: 0.9753 - val_loss: 1.0276 - val_accuracy: 0.8145
Epoch 2/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0461 - accuracy: 0.9848 - val_loss: 0.8992 - val_accuracy: 0.8372
Epoch 3/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0371 - accuracy: 0.9869 - val_loss: 0.9211 - val_accuracy: 0.8233
Epoch 4/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0349 - accuracy: 0.9889 - val_loss: 0.9252 - val_accuracy: 0.8284
Epoch 5/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0245 - accuracy: 0.9926 - val_loss: 0.9997 - val_accuracy: 0.8094
Epoch 6/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0284 - accuracy: 0.9910 - val_loss: 1.3844 - val_accuracy: 0.7786
Epoch 7/100
192/192 [==============================] - 7s 37ms/step - loss: 0.0263 - accuracy: 0.9914 - val_loss: 1.1663 - val_accuracy: 0.8065
Epo

192/192 [==============================] - 14s 71ms/step - loss: 0.0041 - accuracy: 0.9990 - val_loss: 1.3251 - val_accuracy: 0.8277
Epoch 58/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0041 - accuracy: 0.9988 - val_loss: 1.1750 - val_accuracy: 0.8350
Epoch 59/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0064 - accuracy: 0.9980 - val_loss: 1.1882 - val_accuracy: 0.8284
Epoch 60/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 1.2659 - val_accuracy: 0.8248
Epoch 61/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0039 - accuracy: 0.9989 - val_loss: 1.0727 - val_accuracy: 0.8636
Epoch 62/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0029 - accuracy: 0.9995 - val_loss: 1.3986 - val_accuracy: 0.8240
Epoch 63/100
192/192 [==============================] - 14s 71ms/step - loss: 0.0048 - accuracy: 0.9985 - val_loss: 1.3570 - val_accuracy

In [14]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   xtest, ytest, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.8411480784416199
Pruned test accuracy: 0.820403516292572


In [15]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
pruned_keras_file = f'{base_path}/pruned_models_wisdm_gru'
keras.models.save_model(model_for_export, pruned_keras_file , include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

2025-12-15 19:55:29.246069: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:29.246919: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:29.247448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:30.013582: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:30.014796: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:30.015474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:30.672846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:30.673805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:30.674497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:31.399186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:31.400000: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:31.400551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_wisdm_gru/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_wisdm_gru/assets


Saved pruned Keras model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/pruned_models_wisdm_gru


In [16]:
import tensorflow as tf
run_model = tf.function(lambda x: model_for_pruning(x))
BATCH_SIZE = 1
STEPS = 128
INPUT_SIZE = 6
concrete_func = run_model.get_concrete_function(
    tf.TensorSpec([BATCH_SIZE, STEPS, INPUT_SIZE], model_for_pruning.inputs[0].dtype))

# model directory.
MODEL_DIR = f"{base_path}/GRU_Pruning_wisdm"
model_for_pruning.save(MODEL_DIR, save_format="tf", signatures=concrete_func)

2025-12-15 19:55:32.227099: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:32.228105: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:32.228689: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:33.335833: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:33.336898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:33.337448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:34.263551: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:34.264410: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:34.265493: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:35.172661: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-12-15 19:55:35.173455: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-12-15 19:55:35.174005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm/assets


INFO:tensorflow:Assets written to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm/assets


In [17]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
pruned_tflite_model = converter.convert()

pruned_tflite_file = f"{base_path}/models/gru_pruned_wisdm.tflite"

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

2025-12-15 19:55:36.646330: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:36.646409: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:36.646446: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:37.386499: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:37.386575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:37.386607: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:37.890898: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:37.890971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:37.891003: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/gru_pruned_wisdm.tflite


2025-12-15 19:55:38.296687: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 19:55:38.296720: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 19:55:38.297279: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm
2025-12-15 19:55:38.307515: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 19:55:38.307540: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm
2025-12-15 19:55:38.343293: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2025-12-15 19:55:38.353586: I tensorflow/cc/saved_model/loader.cc:231] Restoring 

In [18]:
import tempfile
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [19]:
keras_file = f"{base_path}/models/gru_baseline_wisdm.h5"
keras.models.save_model(base_model, keras_file, include_optimizer=False)

In [20]:
print("Size of gzipped TFlite model: %.2f bytes" % (get_gzipped_model_size(keras_file)))

Size of gzipped TFlite model: 40546.00 bytes


In [21]:
converter = tf.lite.TFLiteConverter.from_saved_model(MODEL_DIR)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

quantized_and_pruned_tflite_file = f"{base_path}/models/gru_pruned_dynamic_wisdm.tflite"

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

2025-12-15 19:55:39.026322: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:39.026399: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:39.026432: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:39.654570: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:39.654644: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:39.654678: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2025-12-15 19:55:40.193140: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_2_grad_concat_split_2_split_dim' with dtype int32
	 [[{{node gradients_split_2_grad_concat_split_2_split_dim}}]]
2025-12-15 19:55:40.193210: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients_split_grad_concat_split_split_dim' with dtype int32
	 [[{{node gradients_split_grad_concat_split_split_dim}}]]
2025-12-15 19:55:40.193245: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Saved quantized and pruned TFLite model to: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/models/gru_pruned_dynamic_wisdm.tflite
Size of gzipped pruned and quantized TFlite model: 51816.00 bytes
Size of gzipped pruned TFlite model: 51787.00 bytes


2025-12-15 19:55:40.665639: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-15 19:55:40.665669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-15 19:55:40.666171: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm
2025-12-15 19:55:40.681406: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2025-12-15 19:55:40.681438: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /s/chopin/k/grad/C837304112/Documents/ESML/final_project/human_activity_recognition/GRU_Pruning_wisdm
2025-12-15 19:55:40.744282: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-15 19:55:40.852386: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bun

In [22]:
def evaluate_model(interpreter):
    in_det  = interpreter.get_input_details()[0]
    out_det = interpreter.get_output_details()[0]
    in_idx  = in_det["index"]
    out_idx = out_det["index"]
    in_dtype = in_det["dtype"]     
    in_scale, in_zero = in_det["quantization"]
    print("dtype", in_dtype)

    prediction_digits = []
    for i, test_image in enumerate(xtest):
        if i % 1000 == 0:
            print(f"Evaluated on {i} results so far.")

        xi = np.expand_dims(test_image, axis=0).astype(np.float32)

        if in_dtype == np.int8:
            xi_q = np.clip(np.round(xi / in_scale + in_zero), -128, 127).astype(np.int8)
            interpreter.set_tensor(in_idx, xi_q)
        elif in_dtype == np.uint8:
            xi_q = np.round(xi / in_scale + in_zero).astype(np.uint8)
            interpreter.set_tensor(in_idx, xi_q)
        else:
            interpreter.set_tensor(in_idx, xi)

        interpreter.invoke()

        yo = interpreter.get_tensor(out_det["index"])[0]
        prediction_digits.append(int(np.argmax(yo)))

    print('\n')
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == ytest).mean()
    return accuracy

In [23]:
interpreter = tf.lite.Interpreter(model_content=quantized_and_pruned_tflite_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Pruned and quantized TFLite test_accuracy:', test_accuracy)
print('Pruned TF test accuracy:', model_for_pruning_accuracy)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


dtype <class 'numpy.float32'>
Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.


Pruned and quantized TFLite test_accuracy: 0.8204035237283319
Pruned TF test accuracy: 0.820403516292572


In [24]:
print("CNN-LSTM Pruned model wisdm size in MB:", os.path.getsize(f"{base_path}/models/gru_pruned_wisdm.tflite") / float(2**20))

CNN-LSTM Pruned model wisdm size in MB: 0.26966094970703125


In [25]:
print("CNN-LSTM Pruned and quantized model wisdm size in MB:", os.path.getsize(f"{base_path}/models/gru_pruned_dynamic_wisdm.tflite") / float(2**20))



CNN-LSTM Pruned and quantized model wisdm size in MB: 0.2696647644042969
